In [1]:
import pandas as pd

df = pd.read_stata('Casen 2017.dta', convert_categoricals = False)

#### Datos MINSAL


##### Casos
* Casos por comuna: https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto1

##### Muertes
* Muertes por comuna: https://github.com/MinCiencia/Datos-COVID19/blob/master/output/producto38/CasosFallecidosPorComuna.csv


#### Datos CASEN

Indicadores: 

###### Educacion
* e1: Sabe leer y escribir?
* e6a: ¿Cuál fue el nivel educacional más alto alcanzado o el nivel educacional actual?
* ??? e8: ¿En qué tipo de institución realizó su educación superior?

##### Trabajo
* o1: La semana pasada, ¿trabajó al menos una hora, sin considerar los quehaceres del hogar?
* o12: ¿Su trabajo o negocio principal es de tipo:
* o15: En su trabajo o negocio principal, ¿usted trabaja como?
* o16: En su trabajo principal, ¿qué tipo de contrato o acuerdo de trabajo tiene?
* o17: En su trabajo principal, ¿tiene contrato de trabajo escrito?
* o25a_min: En un día habitual, ¿cuánto tiempo en total tarda...? minutos
* o25c: ¿Qué medio de transporte utiliza habitualmente para realizar este viaje?
* o28: Se encuentra afiliado a algún sistema previsional (sistema de pensiones)?
* o29: ¿Cotizó durante el mes pasado en algún sistema previsional..?


##### Ingresos
* y21a: Participación núcleo familiar en Chile Solidario
* Y22a. Mes pasado Bono de Protección Familiar CHP 16.634
* y24.Mes pasado Subsidio de Agua Potable (Originales)
* y27a. Cuenta de ahorro o depósito a plazo
* ysub: Subsidios monetarios
* ytot: Ingreso total
* Ab Seite 140: Ingresos? 


ADD: TIEMPO AL TRABAJO / TRANSPORTE PUBLICO

In [1]:
df.head()

NameError: name 'df' is not defined

In [18]:
df_e1 = df.loc[:, ['region', 'comuna', 'ytot']]

df_e1.groupby('comuna').ytot.mean().reset_index().sort_values('ytot', ascending = False)

,comuna,ytot
267,13132,1.987719e+06
249,13114,1.618252e+06
258,13123,1.476416e+06
255,13120,1.081854e+06
248,13113,1.002380e+06
...,...,...
198,9208,1.857542e+05
165,8310,1.854193e+05
163,8308,1.828309e+05
119,7203,1.820784e+05


In [7]:
df.shape

(216439, 804)

In [4]:
df.groupby('region').hogar.count().reset_index()#.sort_values('hogar', ascending = False)


,region,hogar
0,1,10150
1,2,8510
2,3,6961
3,4,9952
4,5,19625
5,6,15862
6,7,15208
7,8,21532
8,9,15481
9,10,12442
